In [2]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

today_date1 = '2024010'
today_date2 = '2023-01-07'

today_date1 = 'reset'

In [29]:
file_name = 'kor_index_ohlcv'
if not os.path.exists(f'data_crawler/cleaning/{file_name}'):
    os.makedirs(f'data_crawler/cleaning/{file_name}')

In [43]:
sql = f"""
select 
*
from `owenchoi-404302.finance_mlops.kor_index_ohlcv`
order by date asc
"""

# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
kor_index_ohlcv = query_job.to_dataframe()

In [50]:
sql = f"""
select 
*
from `owenchoi-404302.finance_mlops.index_code_master`
"""
# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
index_code_master = query_job.to_dataframe()

index_code_master_2 = index_code_master[['index_code', 'index_code_nm','market']].drop_duplicates().reset_index(drop = True)

In [51]:
kor_index_ohlcv_2 = pd.merge(kor_index_ohlcv, index_code_master_2,
        on = 'index_code', 
        how = 'left')

index_code_list = kor_index_ohlcv['index_code'].unique()

In [53]:
df_raw_total = pd.DataFrame()
df_raw_anal_total = pd.DataFrame()

for index_code_nm in index_code_list:
    df_raw = kor_index_ohlcv_2[kor_index_ohlcv_2['index_code'] == index_code_nm].reset_index(drop = True)

    ######################################################################
    # 보조지표
    ######################################################################

    # 이동평균선
    df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
    df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
    df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
    df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

    # 볼린저밴드
    std = df_raw['close'].rolling(20).std(ddof=0)

    df_raw['upper'] = df_raw['MA20'] + 2 * std
    df_raw['lower'] = df_raw['MA20'] - 2 * std

    # MACD
    # MACD
    macd = MACD(close=df_raw['close'],
                window_slow=26,
                window_fast=12,
                window_sign=9)


    df_raw['MACD_DIFF'] = macd.macd_diff()
    df_raw['MACD'] = macd.macd()
    df_raw['MACD_Signal'] = macd.macd_signal()

    # RSI
    df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
    df_raw['변화량'] = df_raw['변화량'].astype('float64')
    df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
    df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

    # welles moving average
    df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

    df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
    df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
    df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']


    ######################################################################
    # 보조지표 분석
    ######################################################################
    df_raw_anal = df_raw[['date','index_code', 'index_code_nm','market', 'close']]

    # 골든크로스
    # 골든 크로스 5-20
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

    # 데드 크로스 5-20
    # 양수에서 음수로 바뀌는 모든 인덱스 찾기
    idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]


    df_raw_anal.loc[:, '5_20_cross'] = '-'
    df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

    df_raw_anal.loc[:, '20_60_cross'] = '-'
    df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'


    # 정배열 역배열
    ascending_sq  = (df_raw['MA5-20'] > 0) & \
    (df_raw['MA20-60'] > 0) & \
    (df_raw['MA60-120'] > 0)

    descending_sq  = (df_raw['MA5-20'] < 0) & \
    (df_raw['MA20-60'] < 0) & \
    (df_raw['MA60-120'] < 0)

    df_raw_anal.loc[:,'array'] = '-'
    df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
    df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'


    # 볼린저밴드
    df_raw['close'] = df_raw['close'].astype('float64')
    down_reg_sq = df_raw['upper'] - df_raw['close']
    top_reg_sq  = df_raw['lower'] - df_raw['close']

    down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
    top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]

    df_raw_anal.loc[:,'Bollinger_band'] = '-'
    df_raw_anal.loc[down_reg,'Bollinger_band'] = '하향회귀(매도)'
    df_raw_anal.loc[top_reg,'Bollinger_band'] = '상향회귀(매수)'


    # MACD
    signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
    signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]

    df_raw_anal.loc[:,'MACD'] = '-'
    df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
    df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'

    # RSI
    down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
    top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]


    df_raw_anal.loc[:,'RSI'] = '-'
    df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
    df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'


    df_raw_total = pd.concat([df_raw_total, df_raw])
    df_raw_anal_total = pd.concat([df_raw_anal_total, df_raw_anal])

    print(index_code_nm)


df_raw_total = df_raw_total.reset_index(drop = True)
df_raw_anal_total = df_raw_anal_total.reset_index(drop = True)

1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1024
1025
1026
1027
1028
1034
1035
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1167
2001
2002
2003
2004
2012
2015
2024
2026
2027
2029
2031
2037
2041
2042
2043
2056
2058
2062
2063
2065
2066
2067
2068
2070
2072
2074
2075
2077
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2181
2182
2183
2184
2203
2212
2213
2214
2215
2216
2217
2218
1150
1182
1224
1227
1232
1244
1894
2189


In [54]:
df_raw_total

,date,open,high,low,close,volume,trading_value,market_cap,index_code,index_code_nm,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
0,2018-01-02 00:00:00+00:00,2474.86,2481.02,2465.94,2479.65,262205431,4786386686165,1613508196692261,1001,코스피,...,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-03 00:00:00+00:00,2484.63,2493.40,2481.91,2486.35,331094681,6019622040484,1617634317727445,1001,코스피,...,NaN,6.70,6.70,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-04 00:00:00+00:00,2502.50,2502.50,2466.45,2466.46,333836328,6896287056563,1604916139690676,1001,코스피,...,NaN,-19.89,0.00,19.89,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-05 00:00:00+00:00,2476.85,2497.52,2475.51,2497.52,308769669,6317518427311,1624970533116195,1001,코스피,...,NaN,31.06,31.06,0.00,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-08 00:00:00+00:00,2510.70,2515.37,2494.18,2513.28,311429324,6404885697956,1634777680824259,1001,코스피,...,NaN,15.76,15.76,0.00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142450,2023-12-28 00:00:00+00:00,1426.70,1438.13,1415.87,1438.13,27445280,1851653389930,115407431270580,2189,코스닥 글로벌,...,31.836115,11.03,11.03,0.00,11.531096,5.939875,66.001459,31.2045,94.663333,-32.951583
142451,2024-01-02 00:00:00+00:00,1427.20,1444.28,1425.96,1440.41,24065022,1163761859030,104286005844470,2189,코스닥 글로벌,...,32.201082,2.28,2.28,0.00,10.870303,5.515598,66.339366,31.9045,95.686333,-30.328333
142452,2024-01-03 00:00:00+00:00,1426.02,1429.66,1402.23,1408.86,17151167,969634094010,101953197267190,2189,코스닥 글로벌,...,32.032287,-31.55,0.00,31.55,10.093853,7.375199,57.781345,30.6310,95.795167,-28.129083
142453,2024-01-04 00:00:00+00:00,1395.22,1422.77,1393.88,1419.89,17269657,925086270830,102745401485250,2189,코스닥 글로벌,...,31.640758,11.03,11.03,0.00,10.160721,6.848399,59.736900,33.3565,94.635500,-26.262750


In [55]:
df_raw_anal_total.tail()

,date,index_code,index_code_nm,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
142450,2023-12-28 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1438.13,-,-,-,-,-,-
142451,2024-01-02 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1440.41,-,-,-,-,-,-
142452,2024-01-03 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1408.86,-,-,-,-,하향돌파(매도),-
142453,2024-01-04 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1419.89,-,-,-,-,-,-
142454,2024-01-05 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1459.75,-,-,-,-,상향돌파(매수),-


In [56]:
df_raw_total_2 = df_raw_total[df_raw_total['date'] > '2023-01-01'].reset_index(drop = True)
df_raw_anal_total_2 = df_raw_anal_total[df_raw_anal_total['date'] > '2023-01-01'].reset_index(drop = True)

In [58]:
table_from_pandas = pa.Table.from_pandas(df_raw_total_2,preserve_index = False)
pq.write_table(table_from_pandas, f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet')

table_from_pandas = pa.Table.from_pandas(df_raw_anal_total_2,preserve_index = False)
pq.write_table(table_from_pandas, f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet')


# Google Storage 적재
source_file_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)


# Google Storage 적재
source_file_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)